In [8]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateFormatter, AutoDateLocator

from os.path import exists

import gdown
import zipfile

from datetime import datetime

### Data Loading

#### Data Frame Columns

 `current_out`: current withdrawn from the box by the consumer <br>
 `current_in`: current supplied to the box from the solar panel <br>
 `current`: `current_out` - `current-in` --> net current that goes in or out of the battery <br>
 `temperature`: temperature in deg C <br>
 `timestamp`: YYYY-MM-DD HH:MM:SS <br>
 `panel_voltage`: voltage of the battery in V <br>
 `state_of_charge_percent`: state of charge of the battery in percent <br>
 `battery_id`: unique identifier of the battery <br>

In [9]:
# Load times series csv file
# Each file contains data from approximately 50 devices
ts1_df = pd.read_csv('..\data/devices1.csv')
ts2_df = pd.read_csv('..\data/devices2.csv')
ts3_df = pd.read_csv('..\data/devices3.csv')
ts4_df = pd.read_csv('..\data/devices4.csv')
 # Concatenate into one big dataframe
ts_df = pd.concat([ts1_df, ts2_df, ts3_df, ts4_df], axis = 0, ignore_index = True)

Downsample the ts_df dataframe by removing rows that contain NaN values.

In [10]:
ts_df_lean = ts_df.dropna()

Add power column and abs(power) column

In [11]:
ts_df_lean["power"] = ts_df_lean["battery_voltage"]*ts_df_lean["current"]
ts_df_lean["abs(power)"] = ts_df_lean["power"].abs()

Convert timestamp to datetime format and add columns for 'year' and 'month'

In [12]:
# convert timestamp column to datetime format. It has useful functionality, such as getting the 'month' or 'year'
ts_df_lean["timestamp"] = pd.to_datetime(ts_df_lean["timestamp"])

In [13]:
#Add year, and month as dataframe columns
year = np.empty((len(ts_df_lean)), dtype=int)
month = np.empty((len(ts_df_lean)), dtype=int)
for i in range(len(ts_df_lean)):
    year[i] = ts_df_lean["timestamp"].iloc[i].year
    month[i] = ts_df_lean["timestamp"].iloc[i].month

In [14]:
ts_df_lean["year"] = year
ts_df_lean["month"] = month

Create two new time-related columns and add them to the dataframe.<br>
Column 'time_battery_sec' contains values which start at 0 for every battery, subsequently counting the operating time of the battery in seconds. <br>
Column 'dt_sec' gives the sampling time step. Will be useful later to calculate time-weighted averages. At the first row of every battery, dt is zero.

In [15]:
# Had to make some conversions back to string to get this to work.
# Some complications were due to numpy changing the format of datetime objects before storing them.
time_battery_sec = np.empty((0,1))
dt_sec = np.empty((0,1))
for battery in range(0,200):
    single_battery_df = ts_df_lean.loc[ts_df_lean['battery_id']==battery]
    # Get the starting timestamp of this battery. This will be time zero.
    # Also place in this more convenient datetime format which will allow subtraction of datetimes.
    datetime_0 = datetime.strptime(str(single_battery_df["timestamp"].iloc[0]), "%Y-%m-%d %H:%M:%S")
    # Initialise numpy array to store battery_time. Loops using pandas would be too slow.
    battery_time = single_battery_df["timestamp"].astype(str).to_numpy()
    for i, entry in enumerate(battery_time):
        datetime_1 = datetime.strptime(str(entry), "%Y-%m-%d %H:%M:%S")
        battery_time[i] = float((datetime_1-datetime_0).total_seconds())
    # Convert to float:
    battery_time = battery_time.astype(float)
    battery_dt = np.zeros((battery_time.shape[0],)) #Useful mostly because it puts a zero at the -1 index for this battery_id.
    battery_dt[0:-1] = battery_time[1:] - battery_time[0:-1]
    time_battery_sec = np.append(time_battery_sec, battery_time)
    dt_sec = np.append(dt_sec, battery_dt)

In [16]:
#Sanity check of dimensions:
print('ts_df_lean.shape: ', ts_df_lean.shape)
print('time_battery_seconds.shape: ', time_battery_sec.shape)
print('dt.shape: ', dt_sec.shape)

ts_df_lean.shape:  (4505554, 15)
time_battery_seconds.shape:  (4505554,)
dt.shape:  (4505554,)


In [17]:
# Add new pandas Series to ts_df_lean
ts_df_lean["time_battery_sec"] = time_battery_sec
ts_df_lean["dt_sec"] = dt_sec

In [18]:
%store ts_df_lean

Stored 'ts_df_lean' (DataFrame)


Some Plots

In [ ]:
# #Plots SoC-binned bar charge of a given dataframe. Specify param (e.g. Mean Temp) for title
# def SoC_binned_bar_chart(battery_id, df, param=''):
#     x = df.columns
#     height = df.iloc[battery_id]
#     f, ax = plt.subplots()
#     ax.bar(x, height, align = 'center')
#     ax.set_title('battery_id = %s. Plot of %s'%(battery_id, param))
#     plt.setp(ax.get_xticklabels(), rotation=30)

### Plotting

In [ ]:
# Basic plot of the data

xtick_locator = AutoDateLocator()
xtick_formatter = AutoDateFormatter(xtick_locator)

start_pidx = 0 
end_pidx = 6000 

dates = matplotlib.dates.date2num(ts_df.loc[ts_df["battery_id"]==0].loc[start_pidx:end_pidx, "timestamp"])
current = ts_df.loc[ts_df["battery_id"]==0].loc[start_pidx:end_pidx, "current"]
voltage = ts_df.loc[ts_df["battery_id"]==0].loc[start_pidx:end_pidx, "battery_voltage"]

fig, host = plt.subplots()
par1 = host.twinx()

p1, = host.plot(dates, current, 'b', label='Current')
p2, = par1.plot(dates, voltage, 'r', label='Voltage')

host.set_ylabel('Current (A)')
par1.set_ylabel('Voltage (V)')
host.set_xlabel('Date')

host.xaxis.set_major_locator(xtick_locator)
host.xaxis.set_major_formatter(xtick_formatter)
fig.autofmt_xdate()

lines = [p1, p2]
host.legend(lines, [l.get_label() for l in lines], loc=1, bbox_to_anchor=(0.995,0.22))
fig.show()